In [18]:
file_path = "../data/farmers-protest-tweets-2021-2-4.json"

In [19]:
import pandas as pd
import json
import time
import datetime
import memory_profiler
#import concurrent

from datetime import datetime
from memory_profiler import memory_usage
from collections import defaultdict, Counter
#from concurrent.futures import ProcessPoolExecutor

### First version of q1_time

In [8]:
def q1_time_pandas(file_path: str):
    # Read the JSON file into a DataFrame
    tweets_df = pd.read_json(file_path, lines=True)

    # Convert the 'date' column to datetime and extract the date
    tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.date

    # Group by 'date' and count tweets, also get the most active user per date
    top_dates_df = tweets_df.groupby('date').agg(
        total_tweets=('date', 'size'),
        most_active_user=('user', lambda x: x.mode()[0]['username'])
    ).sort_values('total_tweets', ascending=False).head(10)

    # Convert the result to the desired format
    top_dates = [(row.name, row['most_active_user']) for _, row in top_dates_df.iterrows()]

    return top_dates

In [9]:
#Executing q1_time_pandas and printing the result
top_dates = q1_time_pandas(file_path)
print(top_dates)

# Timing and memory testing for q1_memory
start_time = time.time()
peak_memory_memory = max(memory_usage((q1_time_pandas, (file_path,))))
end_time = time.time()

print(f"q1_memory - Execution Time: {end_time - start_time} seconds")
print(f"q1_memory - Peak Memory Usage: {peak_memory_memory} MiB")

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
q1_memory - Execution Time: 33.76640963554382 seconds
q1_memory - Peak Memory Usage: 2968.328125 MiB


### Second version of q1_time optimizing the dataframe manipulation

In [10]:
def q1_time_pandas_revised(file_path: str):
    # Load the entire dataset into a DataFrame
    df = pd.read_json(file_path, lines=True)

    # Convert 'date' to datetime and extract the date
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Extract username from the nested 'user' dictionary
    df['username'] = df['user'].apply(lambda x: x['username'])

    # Group by 'date' and count tweets, get the most active user per date
    top_dates_df = df.groupby('date').agg(
        total_tweets=('date', 'size'),
        most_active_user=('username', lambda x: x.mode()[0])
    ).sort_values('total_tweets', ascending=False).head(10).drop(columns=['total_tweets'])

    # Convert to the desired format [(date, most_active_user), ...]
    top_dates = list(top_dates_df.itertuples(index=True, name=None))

    return top_dates

In [11]:
#Executing q1_time_pandas and printing the result
top_dates = q1_time_pandas_revised(file_path)
print(top_dates)

# Timing and memory testing for q1_time
start_time = time.time()
peak_memory_time = max(memory_usage((q1_time_pandas_revised, (file_path,))))
end_time = time.time()

print(f"q1_time_pandas_revised - Execution Time: {end_time - start_time} seconds")
print(f"q1_time_pandas_revised - Peak Memory Usage: {peak_memory_time} MiB")

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
q1_time_pandas_revised - Execution Time: 7.851444482803345 seconds
q1_time_pandas_revised - Peak Memory Usage: 2991.9140625 MiB


### First version of q1_memory using line by line reading and abandoning pandas

In [22]:
def q1_memory(file_path: str):
    tweet_counts = Counter()
    user_activity = defaultdict(Counter)

    # Read and process the JSON file
    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            date = datetime.fromisoformat(tweet['date']).date()
            username = tweet['user']['username']

            # Count tweets per date and track user activity
            tweet_counts[date] += 1
            user_activity[date][username] += 1

    # Identify top 10 dates
    top_dates = tweet_counts.most_common(10)

    # Find the most active user for each top date
    top_dates = [(date, user_activity[date].most_common(1)[0][0]) for date, _ in top_dates]

    return top_dates

In [23]:
#Executing q1_time_pandas and printing the result
top_dates = q1_memory(file_path)
print(top_dates)

# Timing and memory testing for q1_memory
start_time = time.time()
peak_memory_memory = max(memory_usage((q1_memory, (file_path,))))
end_time = time.time()

print(f"q1_memory - Execution Time: {end_time - start_time} seconds")
print(f"q1_memory - Peak Memory Usage: {peak_memory_memory} MiB")

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
q1_memory - Execution Time: 3.816939115524292 seconds
q1_memory - Peak Memory Usage: 59.87109375 MiB


#### q1_memory test with